In [3]:
import cv2
import numpy as np

In [8]:
'''Edge Detector'''
img = cv2.imread("Resources/lena.jpg", 1)

kernel = np.ones((5,5), np.uint8)

imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray, (7,7), 0)
imgCanny = cv2.Canny(img, 100, 100)
imgDilation = cv2.dilate(imgCanny, kernel, iterations=1)
imgEroded = cv2.erode(imgDilation, kernel, iterations=1)

# cv2.imshow("gray image", imgGray)
# cv2.imshow("blur image", imgBlur)
cv2.imshow("canny image",imgCanny)
###Canny Image: getting edges###

cv2.imshow("dilated image",imgDilation)
cv2.imshow("eroded image",imgEroded)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
'''Resizing and Cropping'''
img = cv2.imread("Resources/baboon.jpg",1)
print(img.shape)
#height x width로 보여줌

imgResize = cv2.resize(img, (1000,500))
#argument는 width, height로 받음
print(imgResize.shape)

imgCropped = img[0:200,200:500]
# height, width

cv2.imshow("img", img)
cv2.imshow("resized", imgResize)
cv2.imshow("cropped", imgCropped)

cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 512, 3)
(500, 1000, 3)


In [10]:
'''Warp Perspective'''
img = cv2.imread("Resources/cards.png",1)

width,height = 250,350

point1 = np.float32([[119,219],[287,188],[154,482],[352,440]])
##4 coordniates of image that I want to warp

point2 = np.float32([[0,0],[width,0],[0,height],[width,height]])

matrix = cv2.getPerspectiveTransform(point1,point2)
imgOutput = cv2.warpPerspective(img, matrix, (width,height))


print(point1)
print(point2)
print(img.shape)
print(matrix)
# cv2.imshow("cards", img)
# cv2.imshow("output", imgOutput)

cv2.waitKey(0)

[[119. 219.]
 [287. 188.]
 [154. 482.]
 [352. 440.]]
[[  0.   0.]
 [250.   0.]
 [  0. 350.]
 [250. 350.]]
(480, 640, 3)
[[ 1.69519011e+00 -2.25595643e-01 -1.52322178e+02]
 [ 3.37722338e-01  1.83023719e+00 -4.41010903e+02]
 [ 3.34432201e-05  8.37996876e-04  1.00000000e+00]]


-1

In [1]:
'''Stacking Images Function'''
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

In [55]:
'''Contour Function'''
def getContours(img):
    #image, how to retrieve(e.g. retrieve outer contour)
    contours,hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL
                                          ,cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        print(f'area of contour{area}')
        
        if area>500:
            #noise껴서 이상한 작은그림도 포함 시킬까봐
            cv2.drawContours(imgContour, cnt, -1, (255,0,0), 10)
            # -1은 모든 점들을 뜻해서 결국 line이되고
            #0,1,...숫자를 넣으면 각 지점의 점이된다
            
            peri = cv2.arcLength(cnt, True)
#             True because closed contour
            #perimeter of contour
            print(f'perimeter: {peri}')
            
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            print(f'Polygon is:{len(approx)}')
            
            objCor = len(approx)
            x,y,w,h = cv2.boundingRect(approx)
            
            if objCor ==3: objectType = "Tri"
            elif objCor ==4:
                aspRatio = w/float(h)
                if aspRatio > 0.95 and aspRatio < 1.05: objectType = "Square"
                else: objectType = "Rectangle"
            elif objCor > 4: objectType = "Circle"
            else: objectType = "None"
                
            cv2.rectangle(imgContour, (x,y), (x+w, y+h), (0,255,0),2)
                
            cv2.putText(imgContour, objectType,
                       (x+(w//2)-10,y+(h//2)-10), cv2.FONT_HERSHEY_COMPLEX, 1,
                       (0,0,0), 2)
            # // operator leaves only whole number(내림, x반올림)

In [4]:
'''Joining Images'''

img = cv2.imread("Resources/lena.jpg",1)
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

imgStack = stackImages(0.5, ([img,imgGray,img], [img,img,img]))
#scale, (images to stack)

# imgHor = np.hstack((img,img))
# imgVer = np.vstack((img,img))
# # this only works given same dimension of matrix
# # thus, X for gray + BGR image

# cv2.imshow("horizontal stack", imgHor)
# cv2.imshow("vertical stack", imgVer)

cv2.imshow("imagestack",imgStack)
cv2.waitKey(0)

##### Define function "stackImage" #####


-1

In [16]:
'''Color Detection'''

def empty(a):
    pass

path = 'Resources/butterfly.jpg'

cv2.namedWindow("TrackBars")
cv2.resizeWindow("TrackBars", 640,240)
cv2.createTrackbar("Hue Min", "TrackBars", 0, 179,empty)
#name, window on which to create trackbar, initial value, max value, function
#원래 hue의 max는 360인데 opencv에서는 179까지 허용
cv2.createTrackbar("Hue Max", "TrackBars", 179, 179,empty)
cv2.createTrackbar("Saturation Min", "TrackBars", 0, 255,empty)
cv2.createTrackbar("Saturation Max", "TrackBars", 255, 255,empty)
cv2.createTrackbar("Value Min", "TrackBars", 0, 255,empty)
cv2.createTrackbar("Value Max", "TrackBars", 63, 255,empty)
###원래는 max니까 initial value 255로 세팅해놨는데
### trackbar slide해보면서 나비의 모양(mask)만 뽑아낼수있는 수치들을 찾는다

while(True):
    img = cv2.imread(path)

    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h_min = cv2.getTrackbarPos("Hue Min", "TrackBars")
    h_max = cv2.getTrackbarPos("Hue Max", "TrackBars")
    s_min = cv2.getTrackbarPos("Saturation Min", "TrackBars")
    s_max = cv2.getTrackbarPos("Saturation Max", "TrackBars")
    v_min = cv2.getTrackbarPos("Value Min", "TrackBars")
    v_max = cv2.getTrackbarPos("Value Max", "TrackBars")

    print(h_min, h_max, s_min, s_max, v_min, v_max)
    
    lower = np.array([h_min,s_min,v_min])
    upper = np.array([h_max,s_max,v_max])
    mask = cv2.inRange(imgHSV, lower, upper)
    #filtered out image of that color
    
    imgResult = cv2.bitwise_and(img, img, mask=mask)
    imgStack = stackImages(0.6, ([img,imgHSV],[mask,imgResult]))
#     cv2.imshow("original",img)
#     cv2.imshow("hsv", imgHSV)
#     cv2.imshow("mask", mask)
#     cv2.imshow("result", imgResult)
    ## should show only the color of butterfly
    
    cv2.imshow("stacked",imgStack)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
cv2.destroyAllWindows()

#if used stackImages() function, shown in neater way

0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 63
0 179 0 255 0 

In [56]:
'''Contours/Shape Detection'''
img = cv2.imread("Resources/shape.png")
imgContour = img.copy()

imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray, (7,7), 1)
imgCanny = cv2.Canny(imgBlur, 50,50)
imgBlank = np.zeros_like(img)

getContours(imgCanny)

imgStack = stackImages(0.5, ([img,imgGray,imgBlur], [imgCanny,imgContour,imgBlank]))


cv2.imshow("stacked", imgStack)

cv2.waitKey(0)

area of contour43376.5
perimeter: 1000.9087229967117
Polygon is:3
area of contour52972.0
perimeter: 919.5777697563171
Polygon is:5
area of contour14368.0
perimeter: 485.9209156036377
Polygon is:4
area of contour45031.5
perimeter: 894.811182141304
Polygon is:4
area of contour16263.0
perimeter: 519.6568541526794
Polygon is:4
area of contour23821.5
perimeter: 727.0264731645584
Polygon is:3


-1

In [62]:
'''Viola&Jones'''
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
img = cv2.imread("Resources/lena.jpg")
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = faceCascade.detectMultiScale(imgGray, 1.1, 4)
#image, scale, minimum neighbours

for (x,y,w,h) in faces:
    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)

cv2.imshow("lena", img)
cv2.waitKey(0)

-1